<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/iFish_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**iFish**

https://github.com/Gil-Mor/iFish

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
YOLOv5_model_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/yolov5n_130epch.pt"

# Setup YOLOv5
%cd /content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox #padding

from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


def interference(img_cv2, weight):
    device = 'cpu'
    device = select_device(device)
    model = DetectMultiBackend(weight, device=device, dnn=False)
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #_cv2sz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

    print(f"pred: {pred}")

    return pred


def show_results(img_cv2):
    class_names = {0:"cont", 1:"grav"}
    pred = interference(img_cv2, YOLOv5_model_path)

    # output result
    x1, y1, x2, y2, prob, class_num = torch.round(pred[0][0])

    # probability
    prob = pred[0][0][4].item()


    # class
    class_name = class_names[pred[0][0][5].item()]

    print("診断は %s、確率は%.1f％です。" %(class_name, prob*100))

    # 横幅が640pxになるようにリサイズ
    height, width, _ = img_cv2.shape
    resize_width = 640
    resize_height = int((height / width) * resize_width)
    resize_size = (resize_width, resize_height)
    img_cv2 = cv2.resize(img_cv2, resize_size)

    # calculate coordinates of the bounding box (640*640にpaddingされている分の座標を足す)
    img_height, img_width, _ = img_cv2.shape[:3]
    print(f"img_height: {img_height}, img_width: {img_width}")
    padding_x = (img_height - min(img_width, img_height))/2
    padding_y = (img_width - min(img_width, img_height))/2
    x1 = x1 - padding_x
    y1 = y1 - padding_y
    x2 = x2 - padding_x
    y2 = y2 - padding_y
    print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")


    # draw bounding box
    cv2.rectangle(img_cv2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # show image
    cv2_imshow(img_cv2)


YOLOv5 🚀 v7.0-72-g064365d Python-3.10.6 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.4/225.8 GB disk)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



# Specify the image path
#image_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/train/images/1010.JPG"
#image_path = "/content/drive/MyDrive/Deep_learning/FishEye/sample_img.JPG"
image_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_500px/101.JPG"

# Read the image
img = mpimg.imread(image_path)

# これをグラフ描画前にやって、リセットする
plt.clf()

# Create a figure
plt.figure()

# Display the image along with axis
imgplot = plt.imshow(img)

# Show the colorbar
plt.colorbar()

# Show the grid
plt.grid(True)

# Show the image
plt.show()


In [ ]:
# Required Libraries
import imageio
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import cv2

from google.colab import drive
drive.mount('/content/drive')

# Specify your image path
#image_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/train/images/1010.JPG"
#image_path = "/content/drive/MyDrive/Deep_learning/FishEye/sample_img.JPG"
image_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_500px/101.JPG"


# Fish Eye Filter Function
def get_fish_xn_yn(source_x, source_y, radius, distortion):
    if 1 - distortion*(radius**2) == 0:
        return source_x, source_y
    return source_x / (1 - (distortion*(radius**2))), source_y / (1 - (distortion*(radius**2)))

def apply_fish_eye_filter(image_path, distortion_coefficient=0.5):
    img = imageio.imread(image_path)
    img = img[100:400, :]  # Crop the image as per the requirement
    w, h = img.shape[0], img.shape[1]
    if len(img.shape) == 2:
        bw_channel = np.copy(img)
        img = np.dstack((img, bw_channel))
        img = np.dstack((img, bw_channel))
    if len(img.shape) == 3 and img.shape[2] == 3:
        img = np.dstack((img, np.full((w, h), 255)))
    dstimg = np.zeros_like(img)
    w, h = float(w), float(h)
    for x in range(len(dstimg)):
        for y in range(len(dstimg[x])):
            xnd, ynd = float((2*x - w)/w), float((2*y - h)/h)
            rd = sqrt(xnd**2 + ynd**2)
            xdu, ydu = get_fish_xn_yn(xnd, ynd, rd, distortion_coefficient)
            xu, yu = int(((xdu + 1)*w)/2), int(((ydu + 1)*h)/2)
            if 0 <= xu and xu < img.shape[0] and 0 <= yu and yu < img.shape[1]:
                dstimg[x][y] = img[xu][yu]
    return dstimg.astype(np.uint8)

# Define Distortion Coefficients Range
distortion_coefficients = np.arange(-0.5, 0.6, 0.1)

# # Create Subplots
# fig, axs = plt.subplots(len(distortion_coefficients), figsize=(6, 6 * len(distortion_coefficients)))

# # Apply Fish Eye Filter for each Distortion Coefficient
# for ax, distortion_coefficient in zip(axs, distortion_coefficients):
#     output_img = apply_fish_eye_filter(image_path, distortion_coefficient)
#     ax.imshow(output_img)
#     ax.set_title(f'Fish-eye Filter with Distortion Coefficient = {distortion_coefficient}')
#     ax.axis('off')

# plt.tight_layout()
# plt.show()

# これをグラフ描画前にやって、リセットする
plt.clf()

# Create subplots
fig, axs = plt.subplots(len(distortion_coefficients), 2, figsize=(12, 6 * len(distortion_coefficients)))

# Apply Fish Eye Filter for each Distortion Coefficient
for i, distortion_coefficient in enumerate(distortion_coefficients):
    output_img = apply_fish_eye_filter(image_path, distortion_coefficient)
    axs[i, 0].imshow(output_img)
    axs[i, 0].set_title(f'Fish-eye Filter with Distortion Coefficient = {distortion_coefficient}')
    axs[i, 0].axis('off')

    # Convert image to opencv format
    output_img_cv = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)

    # Perform inference on the distorted image
    show_results(output_img_cv)

    # Display the image with bounding box
    output_img_bb = cv2.cvtColor(output_img_cv, cv2.COLOR_BGR2RGB)
    axs[i, 1].imshow(output_img_bb)
    axs[i, 1].set_title(f'Interference Result')
    axs[i, 1].axis('off')

plt.tight_layout()
plt.show()
